# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re

import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', "stopwords","maxent_ne_chunker", "words"])

from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, ne_chunk


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [6]:
# load data from database
database_filename = "disaster_response_database.db"
engine = create_engine('sqlite:///' + database_filename)
table = database_filename.replace(".db","_table")

df = pd.read_sql_table(table, engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X = df.message.values
y = df.iloc[:,4:].values

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (26216,)
Shape of y: (26216, 35)


### 2. Write a tokenization function to process your text data

### 2.1. Manual operation example before defining function

In [17]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [18]:
# Defining text with a row from X data
text = X[1000]
text

"If I need a US Visa at this moment, what do I need to do, because I don't have anywhere to sleep, my whole family is at my side."

In [19]:
# Normalization : 
# Replace punctuations with " " and make string lowercase
# Replace all punctuations except apostrophes
text = re.sub("[^a-zA-Z0-9']", " ", text.lower())
text

"if i need a us visa at this moment  what do i need to do  because i don't have anywhere to sleep  my whole family is at my side "

In [20]:
# Tokenize: Split to words
#tokenized = nltk.word_tokenize(text)
#print(tokenized)

In [21]:
# Tokenize: Split to words
tokenized = text.split(" ")
tokenized = [word for word in tokenized if word != "" ]
print(tokenized)

['if', 'i', 'need', 'a', 'us', 'visa', 'at', 'this', 'moment', 'what', 'do', 'i', 'need', 'to', 'do', 'because', 'i', "don't", 'have', 'anywhere', 'to', 'sleep', 'my', 'whole', 'family', 'is', 'at', 'my', 'side']


In [22]:
# Remove stop_words
cleaned = [word for word in tokenized if word not in stop_words]
print(cleaned)

['need', 'us', 'visa', 'moment', 'need', 'anywhere', 'sleep', 'whole', 'family', 'side']


In [23]:
# Part of speech tagging
tagged = nltk.pos_tag(cleaned)
tagged

[('need', 'VB'),
 ('us', 'PRP'),
 ('visa', 'VB'),
 ('moment', 'NN'),
 ('need', 'NN'),
 ('anywhere', 'RB'),
 ('sleep', 'JJ'),
 ('whole', 'JJ'),
 ('family', 'NN'),
 ('side', 'NN')]

In [24]:
# Lemmatize
lemmatized = []

tag_dict = {"J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV}

for word, raw_tag in tagged:
    tag = tag_dict.get(raw_tag[0].upper(), wordnet.ADV)
    lemmatized.append(lemmatizer.lemmatize(word, pos = tag))
                          
print(lemmatized)

['need', 'us', 'visa', 'moment', 'need', 'anywhere', 'sleep', 'whole', 'family', 'side']


### 2.2. Define tokenize function

In [25]:
def tokenize(text):
    """
    Funnction tokenize the input text.
    
    Input: text as string
    
    Output: List of words of cleaned text 
    """
    
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # Normalization : 
    # Replace punctuations with " " and make string lowercase
    # Replace all punctuations except apostrophes
    text = re.sub("[^a-zA-Z0-9']", " ", text.lower())
    
    # Tokenize: Split to words
    tokenized = text.split(" ")
    tokenized = [word for word in tokenized if word != "" ]
    
    # Remove stop_words
    cleaned = [word for word in tokenized if word not in stop_words]
    
    # Part of speech tagging
    tagged = nltk.pos_tag(cleaned)
    
    # Lemmatize
    lemmatized = []

    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    for word, raw_tag in tagged:
        tag = tag_dict.get(raw_tag[0].upper(), wordnet.ADV)
        lemmatized.append(lemmatizer.lemmatize(word, pos = tag))

    return lemmatized

In [26]:
# Check the function for 10 item in X: 
for each in X[100:110]:
    print("Original text:",each)
    print("Tokenized text:",tokenize(each))
    print("---")

Original text: The message might be saying that they have been stuck in the presidential palace ( pal ) since the same Tuesday ( as the quake ). They need water. The message says they are not finding a little water. No names, no number of people given.
Tokenized text: ['message', 'might', 'say', 'stick', 'presidential', 'palace', 'pal', 'since', 'tuesday', 'quake', 'need', 'water', 'message', 'say', 'find', 'little', 'water', 'name', 'number', 'people', 'give']
---
Original text: I live in La Plaine, my wife is pregnant, she got injured by falling blocks. She cannot walk. we have not found anyone to help us. We are in the area called Moya
Tokenized text: ['live', 'la', 'plaine', 'wife', 'pregnant', 'get', 'injured', 'fall', 'block', 'cannot', 'walk', 'find', 'anyone', 'help', 'us', 'area', 'call', 'moya']
---
Original text: Our imcomprehensibley of 9 member was repatriated incomprehensible - (Location) Duvivier, Fond Grango
Tokenized text: ['imcomprehensibley', '9', 'member', 'repatria

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 3.2. Define ML Pipeline

In [27]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3 )

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (18351,)
Shape of X_test: (7865,)
Shape of y_train: (18351, 35)
Shape of y_test: (7865, 35)


In [16]:
# fit pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
# prediction
#y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

In [18]:
# pipeline test score
print("Pipeline test score:",pipeline.score(X_test, y_test))

Pipeline test score: 0.238143674507


In [19]:
## Print classification report on train data
#cls_report_train = classification_report(y_train, y_pred_train, target_names = df.iloc[:,4:].columns)
#print("Classification report with Train Data:\n" ,cls_report_train)

# Print classification report on test data
cls_report_test = classification_report(y_test, y_pred_test, target_names = df.iloc[:,4:].columns)
print("Classification report with Test Data:\n" ,cls_report_test)

Classification report with Test Data:
                         precision    recall  f1-score   support

               related       0.79      0.97      0.87      5934
               request       0.78      0.50      0.61      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.61      0.67      3219
          medical_help       0.59      0.27      0.37       638
      medical_products       0.65      0.31      0.42       418
     search_and_rescue       0.53      0.17      0.26       192
              security       0.17      0.03      0.05       144
              military       0.59      0.33      0.43       245
                 water       0.75      0.64      0.69       500
                  food       0.83      0.66      0.73       878
               shelter       0.76      0.56      0.64       705
              clothing       0.61      0.30      0.40       115
                 money       0.47      0.29      0.36       170


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# show parameters of pipeline
for key, value in pipeline.get_params(deep = True).items():
    print("Parameters of {}:".format(key), value)
    print("---------")

Parameters of memory: None
---------
Parameters of steps: [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f6bf9398e18>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('classifier', MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]
---------
Parameters of vect: CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=

In [21]:
# create parameters for grid search
parameters = {
    "classifier__estimator__learning_rate": [0.7], # best parameter of [0.2, 0.5, 0.7]
    "classifier__estimator__n_estimators":[50], # best parameter of [15, 35, 50]
    "vect__ngram_range": [(1,2)], # best parameter of [(1,1),(1,2),(2,2)]
    "vect__max_df": [0.5], # best parameter of [0.2, 0.5, 0.8]
    "tfidf__norm": ["l2"], # best parameter of ["l1","l2"]
    "tfidf__use_idf": [True], # best parameter of [True, False]
    "tfidf__sublinear_tf":[True] # best parameter of [True, False]
}

# initialize and fit grid search
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.7], 'classifier__estimator__n_estimators': [50], 'vect__ngram_range': [(1, 2)], 'vect__max_df': [0.5], 'tfidf__norm': ['l2'], 'tfidf__use_idf': [True], 'tfidf__sublinear_tf': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

In [22]:
print("Best Parameters:", cv.best_params_)

Best Parameters: {'classifier__estimator__learning_rate': 0.7, 'classifier__estimator__n_estimators': 50, 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
print("Test score:", cv.score(X_test, y_test))

Test score: 0.652130267305


In [24]:
# Print classification report on test data
y_pred_test_cv = cv.predict(X_test)

cv_cls_report_test = classification_report(y_test, y_pred_test_cv, target_names = df.iloc[:,4:].columns)
print("CV Classification report with Test Data:\n" ,cv_cls_report_test)

CV Classification report with Test Data:
                         precision    recall  f1-score   support

               related       0.79      0.97      0.87      5934
               request       0.75      0.53      0.62      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.59      0.66      3219
          medical_help       0.58      0.21      0.30       638
      medical_products       0.66      0.27      0.38       418
     search_and_rescue       0.73      0.17      0.27       192
              security       0.12      0.01      0.02       144
              military       0.55      0.22      0.31       245
                 water       0.74      0.68      0.71       500
                  food       0.84      0.64      0.73       878
               shelter       0.77      0.57      0.66       705
              clothing       0.68      0.26      0.38       115
                 money       0.52      0.22      0.31       1

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
# Create custom transformer which calculates text lenght 

class text_length_extractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        extracted_lengths = pd.Series(X).apply(lambda x:len(x))
        return pd.DataFrame(extracted_lengths)

In [30]:
# initialize pipeline 
pipeline = Pipeline([
    ("features", FeatureUnion([
        ("nlp_pipeline", Pipeline([
            ("vect",CountVectorizer()),
            ("tfidf",TfidfTransformer())
        ])),
        
         ("text_len_ext", text_length_extractor())
    ])),
    
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))         
])

In [31]:
# show parameters of pipeline
for key, value in pipeline.get_params(deep = True).items():
    print("Parameters of {}:".format(key), value)
    print("---------")

Parameters of memory: None
---------
Parameters of steps: [('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_r..., smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('text_len_ext', text_length_extractor())],
       transformer_weights=None)), ('classifier', MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]
---------
Parameters of features: FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='ut

In [32]:
# create parameters for grid search
parameters = {
    "classifier__estimator__learning_rate": [0.7], # best parameter of [0.2, 0.5, 0.7]
    "classifier__estimator__n_estimators":[50], # best parameter of [15, 35, 50]
    "features__nlp_pipeline__vect__ngram_range": [(1,2)], # best parameter of [(1,1),(1,2),(2,2)]
    "features__nlp_pipeline__vect__max_df": [0.5], # best parameter of [0.2, 0.5, 0.8]
    "features__nlp_pipeline__tfidf__norm": ["l2"], # best parameter of ["l1","l2"]
    "features__nlp_pipeline__tfidf__use_idf": [True], # best parameter of [True, False]
    "features__nlp_pipeline__tfidf__sublinear_tf":[True] # best parameter of [True, False]
}


In [33]:
# initialize and fit grid search
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.7], 'classifier__estimator__n_estimators': [50], 'features__nlp_pipeline__vect__ngram_range': [(1, 2)], 'features__nlp_pipeline__vect__max_df': [0.5], 'features__nlp_pipeline__tfidf__norm': ['l2'], 'features__nlp_pipeline__tfidf__use_idf': [True], 'features__nlp_pipeline__tfidf__sublinear_tf': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro'

In [34]:
print("Test score:", cv.score(X_test, y_test))

Test score: 0.652741760573


In [35]:
# Print classification report on test data
y_pred_test_cv = cv.predict(X_test)

cv_cls_report_test = classification_report(y_test, y_pred_test_cv, target_names = df.iloc[:,4:].columns)

print("CV Classification report with Test Data:\n" ,cv_cls_report_test)

CV Classification report with Test Data:
                         precision    recall  f1-score   support

               related       0.82      0.93      0.87      5934
               request       0.78      0.54      0.64      1332
                 offer       0.25      0.03      0.05        36
           aid_related       0.74      0.59      0.66      3219
          medical_help       0.60      0.21      0.31       638
      medical_products       0.73      0.26      0.38       418
     search_and_rescue       0.67      0.18      0.28       192
              security       0.13      0.01      0.03       144
              military       0.58      0.27      0.37       245
                 water       0.70      0.69      0.70       500
                  food       0.81      0.70      0.75       878
               shelter       0.80      0.50      0.61       705
              clothing       0.72      0.30      0.42       115
                 money       0.49      0.21      0.30       1

### 9. Export your model as a pickle file

In [5]:
clf_pick  = pickle.dumps('cv.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.